# Imports required

In [82]:
import subprocess, os, re
import pandas as pd

# Get all reports

In [87]:
os.chdir("C:\\Users\\ckumsta\\Documents\\GitHub\\Globalscan6-Plugin")
cmd=['git','log','origin/master','--merges','--first-parent','--date-order','--shortstat','--pretty=format:"%cs %s"']
result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE).stdout.decode('utf-8')

# Convertion into Pandas Dataframe

In [88]:
results = result.split(sep='\n\n')
print(results[1]) # to get an idea of data composition

"2023-08-01 *MINOR* #1990 G7StoneFruits, Speed up open stone defect"
 1 file changed, 5 insertions(+), 19 deletions(-)


In [124]:
# lets look for information types
headers=["year","month","level","hash","subject","fruit","lines_modified"]
years=[]
months=[]
levels=[]
hashes=[]
subjects=[]
fruits=[]
lines_modified=[]
# create regex for informations
reg_description = re.compile(r"(?P<year>\d+)-(?P<month>\d+)-(?P<day>\d+)( )+(\*)?(?P<level>[\w ]+)(\*)?( )+#(?P<hash>\d+)( )?(?P<fruit>\w+)")

good_entries=0
bad_entries=0

#for item in results[:1]:
for item in results[:-1]: # we skip the very first merge which is code creation
    try:
      lines = item.split('\n') # split the item into 2 lines, we want to process the second line only

      if "Merge pull request" in lines[0]:
         #print("UNFORMATTED --> UNUSABLE merge entry")
         bad_entries += 1
         continue

      subject = None
      for line in lines:
        description = reg_description.search(line)
        if description:
          year = description.group('year')
          month = description.group('month')
          level = description.group('level')
          hash = description.group('hash')
          fruit = description.group('fruit').lower()
          if "cherry" in fruit and "tomato" not in fruit:
            fruit="cherry"
          elif "cherry" in fruit:
            fruit="cherrytomato"
          elif "tomat" in fruit:
            fruit="tomato"
          elif "stone" in fruit:
            fruit="stonefruits"
          elif "lemon" in fruit:
            fruit="lemon"
          elif "orange" in fruit:
            fruit="orange"
          else:
            fruit="unknown"
          subject = line     
      if subject == None:
         #print("MALFORMATTED")
         bad_entries += 1
         continue # this item is malformatted and cannot be processed for the stats

      # retrieve insertions, deletion
      modifications_types = [ v.strip() for v in lines[-1].split(',')]
      modifications_types = [ re.search(r"(\d+) (.*)", v) for v in modifications_types]
      modifications = 0
      for modif_type in modifications_types:
        if not modif_type:
          raise Exception(f"What is this line: \"{lines[-1]}\"")
        if "file" not in modif_type.group(2):
           modifications += int(modif_type.group(1))
      
      # finally write down all elements of this entry
      years.append(year)
      months.append(month)
      levels.append(level)
      hashes.append(hash)
      subjects.append(subject)
      fruits.append(fruit)
      lines_modified.append(modifications)
      good_entries += 1
    except Exception as e:
       print("ERROR", e)
       print(item)
df = pd.DataFrame({"year": years, "month": months, "level":levels, "hash": hashes, "fruit": fruits, "modified_lines": lines_modified, "subject": subjects})
print("Bad entries:", bad_entries)
print("Good entries:", good_entries)
df

Bad entries: 100
Good entries: 1159


,year,month,level,hash,fruit,modified_lines,subject
0,2023,08,MINOR,1993,unknown,1,"""2023-08-02 *MINOR* #1993 from MAF-AGROBOTIC/G..."
1,2023,08,MINOR,1990,stonefruits,24,"""2023-08-01 *MINOR* #1990 G7StoneFruits, Speed..."
2,2023,07,MAJOR,1989,stonefruits,93,"""2023-07-31 *MAJOR* #1989 G7StoneFruits add op..."
3,2023,07,MAJOR,1988,stonefruits,72,"""2023-07-28 *MAJOR* #1988 G7StoneFruits, add n..."
4,2023,07,MAJOR,1986,stonefruits,7,"""2023-07-26 *MAJOR* #1986 G7StoneFtruits, skip..."
...,...,...,...,...,...,...,...
1154,2016,09,MINOR,37,unknown,0,"""2016-09-02 *MINOR* #37 move to uEye v4.8"""
1155,2016,09,DEV,36,unknown,2202,"""2016-09-02 *DEV* #36 Implement some Asynchron..."
1156,2016,08,MINOR,33,unknown,192,"""2016-08-05 *MINOR* #33 from Fix-UnhandledExce..."
1157,2016,07,MAJOR,32,unknown,1454,"""2016-07-14 *MAJOR* #32 Update-Updgrader-and-S..."
